In [69]:
import pandas as pd
import numpy as np
#loc = 'D:/University materials/Winter 2018/Applied ML/winter 2018/Assignments/assignment 3/hwk3_datasets/yelp-train.txt'
loc = 'yelp-train.txt'
yelp_train = pd.read_table(loc,header=None,names=['review','label'])

In [70]:
labels = yelp_train.label.unique()
labels.sort()

In [71]:
frq_vect = [] # frequency vector; counts the number of time lables got repeated
for itr in range ( 0,len(labels) ):
    frq_vect = np.append (frq_vect, (sum(yelp_train.label==labels[itr])) ) 
    
No_examples = len(yelp_train.label)
class_prob = frq_vect/No_examples # probability of perticular class occurance

### Using Random Classifier computing F1 score for Yelp train,test,valid data prediction 

In [72]:
def getRandomPred (labels,No_examples,class_prob,true_labels):
    # prediticng class with random classifier and getting F1 score
    pred_labels = np.random.choice(labels, No_examples, p=class_prob)
    from sklearn.metrics import f1_score,accuracy_score

    yelp_f1 = f1_score(true_labels,pred_labels,average='micro')
    yelp_acc = accuracy_score(true_labels,pred_labels)

    print('F1 score',yelp_f1)
    print('Accuracy',yelp_acc)
    return (yelp_f1,yelp_acc)

#### F1 score for Yelp training data

In [73]:
No_examples = len(yelp_train.label)
true_labels = np.asarray(yelp_train.label)
f1_train,train_acc = getRandomPred (labels,No_examples,class_prob,true_labels)

F1 score 0.2671428571428571
Accuracy 0.2671428571428571


#### F1 score for Yelp validaion data

In [74]:
#loc = 'D:/University materials/Winter 2018/Applied ML/winter 2018/Assignments/assignment 3/hwk3_datasets/yelp-valid.txt'
loc = 'yelp-valid.txt'
yelp_valid = pd.read_table(loc,header=None,names=['review','label'])
# for Yelp training data
true_labels = np.asarray(yelp_valid.label)
No_examples = len(yelp_valid.label)
f1_valid,valid_acc = getRandomPred (labels,No_examples,class_prob,true_labels)

F1 score 0.273
Accuracy 0.273


#### F1 score for Yelp testing data

In [75]:
#loc = 'D:/University materials/Winter 2018/Applied ML/winter 2018/Assignments/assignment 3/hwk3_datasets/yelp-test.txt'
loc = 'yelp-test.txt'
yelp_test = pd.read_table(loc,header=None,names=['review','label'])
# for Yelp training data
true_labels = np.asarray(yelp_test.label)
No_examples = len(yelp_test.label)
f1_test,test_acc = getRandomPred (labels,No_examples,class_prob,true_labels)

F1 score 0.2775
Accuracy 0.2775


### Using Majority Classifier computing F1 score for Yelp train,test,valid data prediction 

In [76]:
frq_class = labels[np.argmax(frq_vect)]
frq_class

4

In [77]:
def getMajorityPred(true_labels,No_examples):
    pred_labels = np.repeat(frq_class,len(true_labels))
    from sklearn.metrics import f1_score,accuracy_score
    yelp_f1 = f1_score(true_labels,pred_labels,average='micro')
    yelp_acc = accuracy_score(true_labels,pred_labels)

    print('F1 score',yelp_f1)
    print('Accuracy',yelp_acc)
    return yelp_f1,yelp_acc 

In [78]:
#### F1 score for Yelp training data

In [79]:
true_labels = np.asarray(yelp_train.label)
No_examples = len(yelp_train.label) 
f1_train,train_acc = getMajorityPred(true_labels,No_examples)

F1 score 0.3525714285714286
Accuracy 0.3525714285714286


In [80]:
#### F1 score for Yelp testing data

In [81]:
true_labels = np.asarray(yelp_test.label)
No_examples = len(yelp_test.label) 
f1_test,test_acc = getMajorityPred(true_labels,No_examples)

F1 score 0.351
Accuracy 0.351


In [82]:
#### F1 score for Yelp validation data

In [83]:
true_labels = np.asarray(yelp_valid.label)
No_examples = len(yelp_valid.label) 
f1_valid,valid_acc = getMajorityPred(true_labels,No_examples)

F1 score 0.356
Accuracy 0.356


In [ ]:
###############################################################################################################################



###############################################################################################################################

# Applying naive bayes, Decision tree and Linear SVM

In [3]:
import re
import pandas as pd
import numpy as np

In [4]:
def TextPreprocess(text):   
    text = text.str.lower()
    text = text.str.replace('[^\w\s]','')    
    text = text.str.replace('[0-9]','')
    text = text.str.replace('_','')
    return text

In [5]:
vocab = pd.read_csv('yelp-vocab.txt',header=None,names = ['word','id','frequency'])
#vocabtotext = vocab.word.str.cat(sep=' ')

In [6]:
#loc = 'D:/University materials/Winter 2018/Applied ML/winter 2018/Assignments/assignment 3/hwk3_datasets/yelp-train.txt'
loc = 'yelp-train.txt'
yelp_train = pd.read_table(loc,header=None,names=['review','label'])
yelp_train.review = TextPreprocess(yelp_train.review)

In [7]:
loc = 'yelp-test.txt'
yelp_test = pd.read_table(loc,header=None,names=['review','label'])
yelp_test.review = TextPreprocess(yelp_test.review)

In [8]:
loc = 'valid-train.txt'
yelp_valid = pd.read_table(loc,header=None,names=['review','label'])
yelp_valid.review = TextPreprocess(yelp_valid.review)

In [15]:
def doc_counts(doc): 
    #myDictionary = collections.OrderedDict()
    import collections
    myDictionary = {}  
    myFile = doc
    field = myFile.split()
    frequency=collections.Counter(field) #for bag of frequency
    
    #for line in range(len(field)):
        #myDictionary[line] = [field[line] , line]
        #myDictionary[field[line]] =  line
    return frequency

In [16]:
def generate_sparse_matrix(texts, vocab):
    
    from scipy.sparse import csr_matrix
    """ Generate a sparse matrix from the given texts, using doc_counts function """
    D = len(texts)
    V = len(vocab.word)
        
    mat_bag_data = []
    mat_freq_data = []
    mat_indptr = [0]
    mat_indices = []

    for i,doc in enumerate(texts):
      #  counts,frequency = doc_counts(doc) # counts basically nested list contains words
                                            # frequency hold the number repeatations
        frequency = doc_counts(doc) 
        
        #N = len(counts)  # idk why N requires 
        used = 0
        for word,count in frequency.items():
            if vocab.loc[vocab.word== word].empty:
                # if the word is missing in vocab we skip it
                continue                        
            else:
                index = vocab[vocab.word == word].iloc[0].id - 1  
                # -1 cause our id starts from 1 but in matrix first indice is 0, so we get 10001 example matrix as we skipped 0
                # so -1 so that
                #print(word)
                mat_indices.append(index)
                mat_bag_data.append(1)
                mat_freq_data.append(frequency[word])
                used += 1
        mat_indptr.append(mat_indptr[-1] + used)
        
    mat_bag = csr_matrix((mat_bag_data, mat_indices, mat_indptr), (D,V+1), dtype='int')
    mat_freq = csr_matrix((mat_freq_data, mat_indices, mat_indptr), (D,V+1), dtype='int') 
    # mat_freq has not normalized, normalize it while using it in classifier 
    #mat[:,0] = 1
    
        
    return mat_bag,mat_freq

In [ ]:
#############################################################################################################################
'''
One sprase matrix constructed save it and call the data when its needed as computation is time consuming
'''

############################################################################################################################

### computing sparse matrix of training data

In [17]:
%lsmagic
%time mat_bag,mat_freq = generate_sparse_matrix(yelp_train.review, vocab)

Wall time: 26min 39s


In [18]:
import scipy.sparse
scipy.sparse.save_npz('yelp_train_bag_mat.npz', mat_bag)
scipy.sparse.save_npz('yelp_train_freq_mat.npz', mat_freq)

### computing sparse matrix of testing data
### computing sparse matrix of validation data

In [19]:
%lsmagic
%time test_mat_bag,test_mat_freq = generate_sparse_matrix(yelp_test.review, vocab)
%time valid_mat_bag,valid_mat_freq = generate_sparse_matrix(yelp_valid.review, vocab)

Wall time: 8min 20s
Wall time: 4min 15s


In [20]:
scipy.sparse.save_npz('yelp_test_bag_mat.npz', test_mat_bag)
scipy.sparse.save_npz('yelp_test_freq_mat.npz', test_mat_freq)

scipy.sparse.save_npz('yelp_valid_bag_mat.npz', valid_mat_bag)
scipy.sparse.save_npz('yelp_valid_freq_mat.npz', valid_mat_freq)

In [ ]:
##########################################################################################################################



##########################################################################################################################

### Analysis if Classification Efficiency using bag-of-words

In [1]:
import scipy.sparse
train_mat = scipy.sparse.load_npz('yelp_train_bag_mat.npz')
test_mat = scipy.sparse.load_npz('yelp_test_bag_mat.npz')
valid_mat = scipy.sparse.load_npz('yelp_valid_bag_mat.npz')


In [ ]:
#train_set = yelp_train.iloc[np.random.choice(len(yelp_train.review),20)]       # randomly picking dataset to train
#validation_set = yelp_valid.iloc[np.random.choice(len(yelp_valid.review),20)]      # randomly picking dataset to classify

In [13]:
train_y_true = yelp_train.label
valid_y_true = yelp_valid.label
test_y_true = yelp_test.label

### Applying Naive bayes classifier

In [16]:
# computeing accuracy and F1 score for predicting "training data"
# input sparse matrix data that needs to be predicted
def getClassifierEff (Data,true_y,clf):
    from sklearn.metrics import f1_score,accuracy_score
    y_pred = clf.predict(Data)
    acc = accuracy_score(true_y,y_pred)
    f1 = f1_score(true_y,y_pred,average='micro')
    return f1,acc

In [24]:
# training classifier with "training data"
from sklearn.naive_bayes import BernoulliNB

In [28]:
Hyp = [0, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
yelp_valid_f1 = []
yelp_valid_acc = []
for itr in range(len(Hyp)):
    naive_clf = BernoulliNB(alpha=Hyp[itr]).fit(train_mat, train_y_true)
    f1 , acc = getClassifierEff (valid_mat,valid_y_true,naive_clf)
    yelp_valid_f1.append(f1)
    yelp_valid_acc.append(acc)
   # print('yelp_valid_f1',yelp_valid_f1)

d:\machine learning\python 3.5.2\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [41]:
yelp_valid_f1

[0.38,
 0.38,
 0.382,
 0.386,
 0.387,
 0.394,
 0.401,
 0.408,
 0.42299999999999993,
 0.427,
 0.402]

In [37]:
optim_alpha = Hyp[np.argmax(yelp_valid_f1)]
print('for hyper parameter alpha =',Hyp[np.argmax(yelp_valid_f1)],'we get max F1 score')

for hyper parameter alpha = 0.01 we get max F1 score


In [39]:
naive_clf = BernoulliNB( alpha=optim_alpha ).fit(train_mat, train_y_true)

train_f1 , train_acc = getClassifierEff (train_mat,train_y_true,naive_clf)
test_f1 , test_acc = getClassifierEff (test_mat,test_y_true,naive_clf)

print('yelp_train_f1',train_f1)
print('yelp_test_f1',test_f1)


yelp_train_f1 0.749
yelp_test_f1 0.435


### Applying Decision Tree

In [9]:
from sklearn import tree

class sklearn.tree.DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

In [14]:
crit = ['gini','entropy'] #criterion 
split = ['best','random'] #splitter


In [18]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(crit)):
    for itr2 in range(len(split)):
        tree_clf = tree.DecisionTreeClassifier(criterion = crit[itr1], splitter = split[itr2]).fit(train_mat, train_y_true)
        f1,acc = getClassifierEff (valid_mat,valid_y_true,tree_clf)
        yelp_valid_f1.append(f1)
        yelp_valid_acc.append(acc)
        
        print('yelp_valid_f1',f1,'when we use criterion',crit[itr1],'and splitter',split[itr2])

yelp_valid_f1 0.352 when we use criterion gini and splitter best
yelp_valid_f1 0.347 when we use criterion gini and splitter random
yelp_valid_f1 0.37 when we use criterion entropy and splitter best
yelp_valid_f1 0.361 when we use criterion entropy and splitter random


In [19]:
tree_clf = tree.DecisionTreeClassifier(criterion = 'entropy', splitter = 'best').fit(train_mat, train_y_true)
yelp_train_f1,yelp_train_acc = getClassifierEff (train_mat,train_y_true,tree_clf)
yelp_test_f1,yelp_test_acc = getClassifierEff (test_mat,test_y_true,tree_clf)

print('yelp_train_f1',yelp_train_f1)
print('yelp_test_f1',yelp_test_f1)

yelp_train_f1 1.0
yelp_test_f1 0.333


### Applying Linear SVC

class sklearn.svm.LinearSVC(penalty=’l2’, loss=’squared_hinge’, dual=True, tol=0.0001, C=1.0, multi_class=’ovr’, fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)

In [53]:
from sklearn.svm import LinearSVC
linear_clf = LinearSVC().fit(train_mat, train_y_true)

In [64]:
pen = ['l1','l2']
los = ['squared_hinge'] # gives error for hinge
dul = [False] # gives error for dual = true. 
tolerance = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
C_param = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]


In [67]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(pen)):

    linear_clf = LinearSVC(penalty=pen[itr1] , dual=False).fit(train_mat, train_y_true)
    f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
    yelp_valid_f1.append(f1)
    yelp_valid_acc.append(acc)
            
    print('yelp_valid_f1',f1,'when we use penalty',pen[itr1])

yelp_valid_f1 0.443 when we use penalty l1
yelp_valid_f1 0.44 when we use penalty l2


In [66]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(pen)):
    for itr2 in range(len(tolerance)):
        for itr3 in range(len(C_param)):
            linear_clf = LinearSVC(penalty=pen[itr1],tol=tolerance[itr2],C=C_param[itr3],dual=False).fit(train_mat, train_y_true)
            f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
            yelp_valid_f1.append(f1)
            yelp_valid_acc.append(acc)
            
            print('yelp_valid_f1',f1,'when we use penalty',pen[itr1],'tolerence',tolerance[itr2],'C',C_param[itr3])

yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-10
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-09
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-08
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-07
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-06
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-10 C 1e-05
yelp_valid_f1 0.356 when we use penalty l1 tolerence 1e-10 C 0.0001
yelp_valid_f1 0.372 when we use penalty l1 tolerence 1e-10 C 0.001
yelp_valid_f1 0.425 when we use penalty l1 tolerence 1e-10 C 0.01
yelp_valid_f1 0.496 when we use penalty l1 tolerence 1e-10 C 0.1
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-10
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-09
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-08
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-07
yelp_valid_f1 0.084 when we use penalty l1 tolerence 1e-09 C 1e-

yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-08 C 1e-08
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-08 C 1e-07
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-08 C 1e-06
yelp_valid_f1 0.37899999999999995 when we use penalty l2 tolerence 1e-08 C 1e-05
yelp_valid_f1 0.433 when we use penalty l2 tolerence 1e-08 C 0.0001
yelp_valid_f1 0.495 when we use penalty l2 tolerence 1e-08 C 0.001
yelp_valid_f1 0.502 when we use penalty l2 tolerence 1e-08 C 0.01
yelp_valid_f1 0.473 when we use penalty l2 tolerence 1e-08 C 0.1
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-10
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-09
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-08
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-07
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-07 C 1e-06
yelp_

In [68]:
linear_clf = LinearSVC(penalty='l2',tol=0.1,C=0.01,dual=False).fit(train_mat, train_y_true)
yelp_train_f1,yelp_train_acc = getClassifierEff (train_mat,train_y_true,linear_clf)
yelp_test_f1,yelp_test_acc = getClassifierEff (test_mat,test_y_true,linear_clf)
yelp_valid_f1,yelp_valid_acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
print('yelp_train_f1',yelp_train_f1)
print('yelp_test_f1',yelp_test_f1)
print('yelp_valid_f1',yelp_valid_f1)

yelp_train_f1 0.8092857142857144
yelp_test_f1 0.502
yelp_valid_f1 0.508


### for combination of penalty='l2' , loss='hinge',dual=True

In [84]:
pen = ['l2']
los = ['hinge'] # gives error for hinge
dul = [True] # gives error for dual = true. 
tolerance = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
C_param = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

In [85]:
yelp_valid_f1 = []
yelp_valid_acc = []
for itr1 in range(len(pen)):
    for itr2 in range(len(tolerance)):
        for itr3 in range(len(C_param)):
            linear_clf = LinearSVC(penalty=pen[itr1],tol=tolerance[itr2],C=C_param[itr3],dual=False).fit(train_mat, train_y_true)
            f1 , acc = getClassifierEff (valid_mat,valid_y_true,linear_clf)
            yelp_valid_f1.append(f1)
            yelp_valid_acc.append(acc)
            
            print('yelp_valid_f1',f1,'when we use penalty',pen[itr1],'tolerence',tolerance[itr2],'C',C_param[itr3])

yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-10
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-09
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-08
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-07
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-10 C 1e-06
yelp_valid_f1 0.37899999999999995 when we use penalty l2 tolerence 1e-10 C 1e-05
yelp_valid_f1 0.433 when we use penalty l2 tolerence 1e-10 C 0.0001
yelp_valid_f1 0.495 when we use penalty l2 tolerence 1e-10 C 0.001
yelp_valid_f1 0.502 when we use penalty l2 tolerence 1e-10 C 0.01
yelp_valid_f1 0.473 when we use penalty l2 tolerence 1e-10 C 0.1
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-09 C 1e-10
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-09 C 1e-09
yelp_valid_f1 0.35500000000000004 when we use penalty l2 tolerence 1e-09 C 1e-08
yelp_

In [94]:
linear_clf = LinearSVC(penalty='l2',tol=0.1,C=0.01,loss='hinge',dual=True).fit(train_mat, train_y_true)
yelp_train_f1,yelp_train_acc = getClassifierEff (train_mat,train_y_true,linear_clf)
yelp_test_f1,yelp_test_acc = getClassifierEff (test_mat,test_y_true,linear_clf)
print('yelp_train_f1',yelp_train_f1)
print('yelp_test_f1',yelp_test_f1)

yelp_train_f1 0.7461428571428571
yelp_test_f1 0.506
